# 🛰️ Interactive Spacecraft Attitude Control Simulator

**Your Mission:** Stabilize a tumbling satellite using a PD controller. Adjust the gains `Kp` and `Kd` to make the satellite reach its target orientation as quickly and smoothly as possible!

**Controls:**
- `Kp` (Proportional Gain): How strongly to push toward the target
- `Kd` (Derivative Gain): How much to dampen the motion (prevent overshooting)
- Target Roll/Pitch/Yaw: Where you want the satellite to point

### How it Works: A Step-by-Step Guide

This notebook simulates a real-world control problem. Let's break down how it works, cell by cell.

In [ ]:
# Import all necessary libraries
import numpy as np
from scipy.integrate import solve_ivp
from scipy.spatial.transform import Rotation as R
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
from mpl_toolkits.mplot3d.art3d import Poly3DCollection
from ipywidgets import interact, FloatSlider, Checkbox
from IPython.display import HTML, display

# Spacecraft properties
I = np.diag([10.0, 20.0, 30.0])  # Inertia tensor [kg*m^2]
I_inv = np.linalg.inv(I)

# Initial conditions: tumbling satellite
q_initial = R.from_euler('xyz', [10, -20, 30], degrees=True).as_quat()
q_initial = np.array([q_initial[3], q_initial[0], q_initial[1], q_initial[2]]) # to [w,x,y,z]
omega_initial = np.array([0.1, -0.2, 0.3])  # Initial angular velocity [rad/s]

# Simulation time
t_span = [0, 60]  # seconds

---
### Cell 2: Setup and Initial Conditions

This cell sets the stage.

1.  **Imports**: It brings in all the necessary tools (`numpy` for math, `matplotlib` for plotting, etc.).
2.  **Spacecraft Properties**: We define the satellite's `I` (Inertia Tensor), which is like its weight and how that weight is distributed. A higher inertia means it's harder to start or stop its rotation.
3.  **Initial State**: We set the satellite's starting conditions:
    *   `q_initial`: The initial orientation. We start it off-kilter.
    *   `omega_initial`: The initial spin. We start it tumbling.
    *   `t_span`: We tell the simulation to run for 60 seconds.

In [ ]:
# Core simulation functions
def get_error_quaternion(q_target, q_current):
    """Calculate error between current and target quaternions."""
    # Simple quaternion error calculation
    q_target_conj = np.array([q_target[0], -q_target[1], -q_target[2], -q_target[3]])
    q_error = np.array([
        q_current[0]*q_target_conj[0] - q_current[1]*q_target_conj[1] - q_current[2]*q_target_conj[2] - q_current[3]*q_target_conj[3],
        q_current[0]*q_target_conj[1] + q_current[1]*q_target_conj[0] + q_current[2]*q_target_conj[3] - q_current[3]*q_target_conj[2],
        q_current[0]*q_target_conj[2] - q_current[1]*q_target_conj[3] + q_current[2]*q_target_conj[0] + q_current[3]*q_target_conj[1],
        q_current[0]*q_target_conj[3] + q_current[1]*q_target_conj[2] - q_current[2]*q_target_conj[1] + q_current[3]*q_target_conj[0]
    ])
    if q_error[0] < 0:
        q_error = -q_error
    return q_error

def spacecraft_dynamics(t, y, q_target, Kp, Kd):
    """Complete spacecraft dynamics with PD control."""
    q = y[:4] / np.linalg.norm(y[:4])  # Normalize quaternion
    omega = y[4:]
    
    # Calculate control torque
    q_err = get_error_quaternion(q_target, q)
    torque = -Kp * q_err[1:] - Kd * omega  # PD control law
    
    # Quaternion kinematics
    omega_matrix = np.array([
        [0, -omega[0], -omega[1], -omega[2]],
        [omega[0], 0, omega[2], -omega[1]],
        [omega[1], -omega[2], 0, omega[0]],
        [omega[2], omega[1], -omega[0], 0]
    ])
    q_dot = 0.5 * omega_matrix @ q
    
    # Rotational dynamics (Euler's equations)
    omega_dot = I_inv @ (torque - np.cross(omega, I @ omega))
    
    return np.concatenate((q_dot, omega_dot))

def run_simulation(q_target, Kp, Kd):
    """Run the attitude control simulation."""
    y0 = np.concatenate((q_initial, omega_initial))
    t_eval = np.linspace(t_span[0], t_span[1], 300)
    
    sol = solve_ivp(spacecraft_dynamics, t_span, y0, t_eval=t_eval, 
                    args=(q_target, Kp, Kd), method='RK45')
    return sol.t, sol.y

---
### Cell 3: The Physics Engine (Core Simulation)

This is the brain of the simulator. It contains the laws of physics.

-   `get_error_quaternion`: Calculates the difference between where the satellite *is* and where you *want it to be*. This is the "error" the controller needs to fix.
-   `spacecraft_dynamics`: This is the core physics model. At every moment in time, it:
    1.  Calculates the **control torque** using the PD controller (`-Kp * error - Kd * spin`). This is the "push" needed to correct the satellite.
    2.  Applies **Euler's Equations of Motion** to figure out how that torque will change the satellite's spin (`omega_dot`).
    3.  Calculates how the spin will change the satellite's orientation (`q_dot`).
-   `run_simulation`: This function uses SciPy's `solve_ivp` to run the simulation, calling `spacecraft_dynamics` repeatedly to predict the satellite's state over 60 seconds.

In [ ]:
# 3D Animation function
def create_3d_animation(t, y):
    """Create 3D animation of spacecraft attitude."""
    q_hist = y[:4, :]
    
    fig = plt.figure(figsize=(10, 8))
    fig.patch.set_facecolor('black')
    ax = fig.add_subplot(111, projection='3d')
    
    # More realistic spacecraft model: body and two solar panels
    # Body
    body_verts = np.array([[-0.5, -0.5, -0.5], [0.5, -0.5, -0.5], [0.5, 0.5, -0.5], [-0.5, 0.5, -0.5],
                           [-0.5, -0.5, 0.5], [0.5, -0.5, 0.5], [0.5, 0.5, 0.5], [-0.5, 0.5, 0.5]])
    body_faces = [[0, 1, 2, 3], [4, 7, 6, 5], [0, 4, 5, 1], [2, 6, 7, 3], [1, 5, 6, 2], [0, 3, 7, 4]]

    # Solar Panel 1
    panel1_verts = np.array([[-2.5, 0.7, 0], [-0.5, 0.7, 0], [-0.5, 2.2, 0], [-2.5, 2.2, 0]])
    
    # Solar Panel 2
    panel2_verts = np.array([[-2.5, -2.2, 0], [-0.5, -2.2, 0], [-0.5, -0.7, 0], [-2.5, -0.7, 0]])

    def update(frame):
        ax.clear()
        # Set space-like background and text colors
        ax.set_facecolor('black')
        ax.set_title("Spacecraft 3D Orientation", fontsize=16, color='white')
        
        q = q_hist[:, frame]
        r = R.from_quat([q[1], q[2], q[3], q[0]])
        
        # Rotate and draw body
        body_rotated = r.apply(body_verts)
        body_collection = Poly3DCollection([body_rotated[f] for f in body_faces], 
                                           facecolors='silver', edgecolors='black', alpha=0.9)
        ax.add_collection3d(body_collection)
        
        # Rotate and draw solar panels
        panel1_rotated = r.apply(panel1_verts)
        panel2_rotated = r.apply(panel2_verts)
        panel_collection = Poly3DCollection([panel1_rotated, panel2_rotated], 
                                            facecolors='midnightblue', edgecolors='lightblue', alpha=0.9)
        ax.add_collection3d(panel_collection)
        
        # Draw target orientation axes
        ax.quiver(0, 0, 0, 3, 0, 0, color='w', linestyle='--', linewidth=1, arrow_length_ratio=0.1, label='Target X')
        ax.quiver(0, 0, 0, 0, 3, 0, color='w', linestyle='--', linewidth=1, arrow_length_ratio=0.1, label='Target Y')
        ax.quiver(0, 0, 0, 0, 0, 3, color='w', linestyle='--', linewidth=1, arrow_length_ratio=0.1, label='Target Z')

        # Set plot limits and labels with white text
        ax.set_xlim([-3.5, 3.5])
        ax.set_ylim([-3.5, 3.5])
        ax.set_zlim([-3.5, 3.5])
        ax.set_xlabel('X-axis', color='white')
        ax.set_ylabel('Y-axis', color='white')
        ax.set_zlabel('Z-axis', color='white')
        
        # Set tick and pane colors to white
        ax.tick_params(axis='x', colors='white')
        ax.tick_params(axis='y', colors='white')
        ax.tick_params(axis='z', colors='white')
        ax.xaxis.pane.set_edgecolor('w')
        ax.yaxis.pane.set_edgecolor('w')
        ax.zaxis.pane.set_edgecolor('w')
        
        # Set grid color
        ax.grid(color='gray', linestyle='--', linewidth=0.5, alpha=0.5)
        
        legend = ax.legend(loc='upper right')
        plt.setp(legend.get_texts(), color='white')
        
        ax.view_init(elev=20, azim=45) # Set a nice viewing angle

    frames = np.linspace(0, len(t)-1, 60, dtype=int)
    ani = FuncAnimation(fig, update, frames=frames, interval=100, repeat=True)
    plt.close(fig)
    return HTML(ani.to_html5_video())

---
### Cell 4: The 3D Animation

This cell takes the raw data from the simulation and turns it into a visual.

-   It defines a simple 3D model of the satellite (a box).
-   The `update` function is called for each frame of the animation. It takes the satellite's orientation at that specific time and rotates the 3D model to match.
-   `FuncAnimation` stitches all these frames together into a smooth video, which is then displayed as an HTML5 video in the notebook.

In [ ]:
# Interactive Control Panel
@interact(
    Kp=FloatSlider(value=2.0, min=0.1, max=10.0, step=0.1, description='Kp Gain:'),
    Kd=FloatSlider(value=4.0, min=0.1, max=10.0, step=0.1, description='Kd Damping:'),
    target_roll=FloatSlider(value=0, min=-90, max=90, step=10, description='Target Roll:'),
    target_pitch=FloatSlider(value=0, min=-90, max=90, step=10, description='Target Pitch:'),
    target_yaw=FloatSlider(value=0, min=-180, max=180, step=10, description='Target Yaw:'),
    show_plots=Checkbox(value=True, description='Show Detailed Plots')
)
def interactive_simulation(Kp, Kd, target_roll, target_pitch, target_yaw, show_plots):
    # Convert target angles to quaternion
    q_target = R.from_euler('xyz', [target_roll, target_pitch, target_yaw], degrees=True).as_quat()
    q_target = np.array([q_target[3], q_target[0], q_target[1], q_target[2]])  # [w,x,y,z]
    
    print(f"🎮 Controller: Kp={Kp:.1f}, Kd={Kd:.1f}")
    print(f"🎯 Target: Roll={target_roll}°, Pitch={target_pitch}°, Yaw={target_yaw}°")
    
    # Run simulation
    t, y = run_simulation(q_target, Kp, Kd)
    
    # Show 3D animation
    print("🛰️ Generating spacecraft animation...")
    animation = create_3d_animation(t, y)
    display(animation)
    
    # Process data for plots
    q_hist, omega_hist = y[:4, :], y[4:, :]
    euler_hist = np.array([R.from_quat([q[1], q[2], q[3], q[0]]).as_euler('xyz', degrees=True) 
                          for q in q_hist.T])
    
    # Optional plotting
    if show_plots:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 8), sharex=True)
        
        # Attitude plot
        ax1.plot(t, euler_hist[:, 0], 'r-', label='Roll', linewidth=2)
        ax1.plot(t, euler_hist[:, 1], 'g-', label='Pitch', linewidth=2)
        ax1.plot(t, euler_hist[:, 2], 'b-', label='Yaw', linewidth=2)
        ax1.axhline(target_roll, color='r', linestyle='--', alpha=0.7, label='Target Roll')
        ax1.axhline(target_pitch, color='g', linestyle='--', alpha=0.7, label='Target Pitch')
        ax1.axhline(target_yaw, color='b', linestyle='--', alpha=0.7, label='Target Yaw')
        ax1.set_ylabel('Angle (degrees)')
        ax1.set_title('Spacecraft Attitude vs Time')
        ax1.legend()
        ax1.grid(True)
        
        # Angular velocity plot
        ax2.plot(t, omega_hist[0, :], 'r-', label='ωx', linewidth=2)
        ax2.plot(t, omega_hist[1, :], 'g-', label='ωy', linewidth=2)
        ax2.plot(t, omega_hist[2, :], 'b-', label='ωz', linewidth=2)
        ax2.set_xlabel('Time (seconds)')
        ax2.set_ylabel('Angular Velocity (rad/s)')
        ax2.set_title('Angular Velocity vs Time')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plt.show()
    
    # Performance summary
    final_error = np.linalg.norm(euler_hist[-1] - [target_roll, target_pitch, target_yaw])
    final_omega = np.linalg.norm(omega_hist[:, -1])
    print(f"📊 Final attitude error: {final_error:.1f}°")
    print(f"📊 Final angular velocity: {final_omega:.3f} rad/s")
    if final_error < 5 and final_omega < 0.1:
        print("🎉 Excellent! Satellite is stable!")
    elif final_error < 15:
        print("👍 Good job! Try fine-tuning the gains for better performance.")
    else:
        print("🔧 Needs work. Try adjusting Kp and Kd values.")

---
### Cell 5: The Interactive Control Panel

This is where you, the user, take control.

-   `@interact`: This powerful command from `ipywidgets` automatically creates sliders for the function arguments.
-   `interactive_simulation`: This function ties everything together. **Every time you move a slider:**
    1.  The new values for `Kp`, `Kd`, and the target angles are passed to the function.
    2.  It calls `run_simulation` to get the new trajectory.
    3.  It calls `create_3d_animation` to generate the new video.
    4.  It displays the animation, the 2D plots, and the final performance summary.

This creates a feedback loop: **You change the controls -> The simulation runs -> You see the results -> You adjust the controls again.**